In [1]:
import gym
import random
import numpy as np
from collections import deque
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print("Gym:", gym.__version__)
print("TensorflowL:", tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
Gym: 0.17.3
TensorflowL: 2.1.0


https://github.com/openai/gym/wiki/CartPole-v0

https://youtu.be/MqZmwQoOXw4

https://arxiv.org/abs/1511.05952

In [2]:
env_name = "CartPole-v0"
env = gym.make(env_name)

# env is now our handle for interacting with the environement

# Prioritized Experience Replay

Prioritize on error = q(s, a) - q_target
add offset so zero error doesn't mean zero priority (we still want to train on these experiences later)
Pr(i) = p_i^a/sum(p^a)  where a <- [0, 1] is a dampening factor.

Loss fn is the squared error.

For each lay\er, update by adding learning_rate * derivatives of the loss fn * a weight factor.

Importance Sampling Weights: factor = 1/buffer size * 1/priority probability.
- Reduce step size for higher probablility experiences.
- Wrap the weight factor in an exponent b, which starts low and increases over time to 1.

In [3]:
class QNetwork():
    def __init__(self, state_dim, action_size, tau=0.01):
        tf.reset_default_graph()
        self.state_in = tf.placeholder(tf.float32, shape=[None, *state_dim])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.q_target_in = tf.placeholder(tf.float32, shape=[None])
        self.importance_in = tf.placeholder(tf.float32, shape=[None])
        action_one_hot = tf.one_hot(self.action_in, depth=action_size)
        
        self.q_state_local = self.build_model(action_size, 'local')
        self.q_state_target = self.build_model(action_size, 'target')

        # The loss function here uses the local net
        self.q_state_action = tf.reduce_sum(tf.multiply(self.q_state_local, action_one_hot), axis=1)
        self.error = self.q_state_action - self.q_target_in
        self.loss = tf.reduce_mean(tf.multiply(tf.square(self.error), self.importance_in))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss)
        
        self.local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='local')
        self.target_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='target')
        
        # Update the target net with corresponding variables in local net
        self.updater = tf.group([tf.assign(t, t + tau*(l-t)) for t,l in zip(self.target_vars, self.local_vars)])

        
    # call the build_model method twice in constructor, one for local net, one for target net.
    def build_model(self, action_size, scope):
        with tf.variable_scope(scope):
            hidden1 = tf.layers.dense(self.state_in, 100, activation=tf.nn.relu)
            q_state = tf.layers.dense(hidden1, action_size, activation=None)
            return q_state
        
    def update_model(self, session, state, action, q_target, importance):
        feed = {self.state_in: state, self.action_in: action, self.q_target_in: q_target, self.importance_in: importance}
        error, _, _ = session.run([self.error, self.optimizer, self.updater], feed_dict=feed)
        return error
        
    def get_q_state(self, session, state, use_target=False):
        q_state_op = self.q_state_target if use_target else self.q_state_local
        q_state = session.run(q_state_op, feed_dict={self.state_in: state})
        return q_state

In [4]:
class PrioritizedReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = deque(maxlen=maxlen)
        self.priorities = deque(maxlen=maxlen)
        
    def add(self, experience):
        self.buffer.append(experience)
        self.priorities.append(max(self.priorities, default=1))
        
    def get_probabilities(self, priority_scale):
        scaled_priorities = np.array(self.priorities) ** priority_scale
        sample_probabilities = scaled_priorities / sum(scaled_priorities)
        return sample_probabilities
    
    def get_importance(self, probabilities):
        importance = 1/len(self.buffer) * 1/probabilities
        importance_normalized = importance / max(importance)
        return importance_normalized
    
    def sample(self, batch_size, priority_scale=1.0):
        sample_size = min(len(self.buffer), batch_size)
        sample_probs = self.get_probabilities(priority_scale)
        sample_indices = random.choices(range(len(self.buffer)), k=sample_size, weights=sample_probs)
        samples = np.array(self.buffer)[sample_indices]
        importance = self.get_importance(sample_probs[sample_indices])
        return map(list, zip(*samples)), importance
    
    def set_priorities(self, indices, errors, off_set=0.1):
        for i,e in zip(indices, errors):
            self.priorities[i] = abs(e) + offset

In [5]:
class DoubleDQNAgent():
    def __init__(self, env):
        self.state_dim = env.observation_space.shape
        self.action_size = env.action_space.n
        self.q_network = QNetwork(self.state_dim, self.action_size)
        self.replay_buffer = PrioritizedReplayBuffer(maxlen=100000)
        self.gamma = 0.97
        self.eps = 1.0
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def get_action(self, state):
        q_state = self.q_network.get_q_state(self.sess, [state])
        action_greedy = np.argmax(q_state)
        action_random = np.random.randint(self.action_size)
        action = action_random if random.random() < self.eps else action_greedy        
        return action
    
    def train(self, state, action, next_state, reward, done):
        self.replay_buffer.add((state, action, next_state, reward, done))
        (states, actions, next_states, rewards, dones), importance = self.replay_buffer.sample(50)
        
        # Use local net to get action using argmax
        next_actions = np.argmax(self.q_network.get_q_state(self.sess, next_states, use_target=False), axis=1)
        q_next_states = self.q_network.get_q_state(self.sess, next_states, use_target=True)
        q_next_states[dones] = np.zeros([self.action_size])
        q_next_states_next_actions = q_next_states[np.arange(next_actions.shape[0]), next_actions]
        q_targets = rewards + self.gamma * q_next_states_next_actions
        errors = self.q_network.update_model(self.sess, states, actions, q_targets, importance**(1-self.eps))
        
        if done: self.eps = max(0.1, 0.99*self.eps)
    
    def __del__(self):
        self.sess.close()

In [ ]:
agent = DoubleDQNAgent(env)
num_episodes = 400


for ep in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train(state, action, next_state, reward, done)
        env.render()
        total_reward += reward
        state = next_state
        
    print("Episode: {}, total_reward: {:.2f}".format(ep, total_reward))

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


C:\Users\15309\AppData\Local\Continuum\anaconda3\envs\baselines\lib\site-packages\ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Episode: 0, total_reward: 18.00
Episode: 1, total_reward: 31.00
Episode: 2, total_reward: 25.00
Episode: 3, total_reward: 16.00
Episode: 4, total_reward: 19.00
Episode: 5, total_reward: 10.00
Episode: 6, total_reward: 35.00
Episode: 7, total_reward: 15.00
Episode: 8, total_reward: 20.00
Episode: 9, total_reward: 10.00
Episode: 10, total_reward: 20.00
Episode: 11, total_reward: 14.00
Episode: 12, total_reward: 28.00
Episode: 13, total_reward: 10.00
Episode: 14, total_reward: 10.00
Episode: 15, total_reward: 13.00
Episode: 16, total_reward: 23.00
Episode: 17, total_reward: 12.00
Episode: 18, total_reward: 20.00
Episode: 19, total_reward: 19.00
Episode: 20, total_reward: 13.00
Episode: 21, total_reward: 22.00
Episode: 22, total_reward: 14.00
Episode: 23, total_reward: 20.00
Episode: 24, total_reward: 9.00
Episode: 25, total_reward: 11.00
Episode: 26, total_reward: 12.00
Episode: 27, total_reward: 15.00
Episode: 28, total_reward: 21.00
Episode: 29, total_reward: 17.00
Episode: 30, total_re